In [1]:
import pandas as pd
import numpy as np

In [21]:
#read log generated from MIMIC-IV Event Log Generation Tool
transfers = pd.read_csv("EL_2.csv")

In [22]:
transfers = transfers.rename(columns={"case:hadm_id":"hadm_id", "time:timestamp":"intime", "concept:name":"careunit"})

In [23]:
transfers = transfers.sort_values(["hadm_id", "intime"])

In [24]:
transfers = transfers.loc[~transfers["hadm_id"].isna()]

In [25]:
transfers.reset_index(inplace=True)

In [26]:
transfers.drop("index", axis=1, inplace=True)

In [27]:
hf_t = transfers

In [28]:
hf_t["intime"] = pd.to_datetime(hf_t["intime"])
hf_t["outtime"] = pd.to_datetime(hf_t["outtime"])

In [29]:
hf_t.loc[hf_t["outtime"].isna(), "outtime"] = hf_t["intime"] + pd.Timedelta(seconds=1)

In [30]:
hf_t.reset_index(inplace=True)
hf_t.drop("index", axis=1, inplace=True)

In [34]:
hf_t.loc[(hf_t["careunit"].isna()) & (hf_t["eventtype"] == "discharge"), "careunit"] = "Discharged"
hf_t.loc[hf_t["eventtype"] == "discharge", "careunit"] = "Discharged"

In [36]:
hf_t.loc[(hf_t["careunit"].isna()), "careunit"] = "Unknown"

In [37]:
hf_t

,Unnamed: 0,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,Glucose_valuenum,Potassium_valuenum,...,case:language,case:marital_status,case:race,case:edregtime,case:edouttime,case:hospital_expire_flag,case:gender,case:age,case:icd_code,case:drg_code
0,0,18810660,20000694,30748315,ED,Emergency Department,2144-08-31 13:34:00,2144-08-31 18:56:00,NaN,NaN,...,ENGLISH,WIDOWED,WHITE,2144-08-31 13:34:00,2144-08-31 18:56:00,0.0,F,73.0,"['N179 ', 'J811 ', 'Z940 ', 'B258 ', '...","['469', '683']"
1,1,18810660,20000694,32094658,admit,Transplant,2144-08-31 18:56:00,2144-09-04 20:41:55,115.750000,5.125000,...,ENGLISH,WIDOWED,WHITE,2144-08-31 13:34:00,2144-08-31 18:56:00,0.0,F,73.0,"['N179 ', 'J811 ', 'Z940 ', 'B258 ', '...","['469', '683']"
2,2,18810660,20000694,36354743,transfer,Transplant,2144-09-04 20:41:55,2144-09-05 15:15:04,120.000000,5.000000,...,ENGLISH,WIDOWED,WHITE,2144-08-31 13:34:00,2144-08-31 18:56:00,0.0,F,73.0,"['N179 ', 'J811 ', 'Z940 ', 'B258 ', '...","['469', '683']"
3,3,18810660,20000694,34992030,discharge,Discharged,2144-09-05 15:15:04,2144-09-05 15:15:05,NaN,NaN,...,ENGLISH,WIDOWED,WHITE,2144-08-31 13:34:00,2144-08-31 18:56:00,0.0,F,73.0,"['N179 ', 'J811 ', 'Z940 ', 'B258 ', '...","['469', '683']"
4,4,11082580,20000769,32839076,admit,Labor & Delivery,2167-08-31 13:19:29,2167-09-02 18:00:07,118.000000,4.800000,...,ENGLISH,MARRIED,ASIAN,NaN,NaN,0.0,F,43.0,"['O4212 ', 'N179 ', 'O721 ', 'O1414 ', '...","['540', '787']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143181,143181,10053207,29999444,35907476,transfer,Emergency Department Observation,2199-12-14 19:16:36,2199-12-14 19:52:26,NaN,NaN,...,ENGLISH,WIDOWED,WHITE,2199-12-14 13:18:00,2199-12-14 21:29:00,0.0,F,59.0,"['0389 ', '5845 ', '59010 ', '29680 ', '...","['720', '871']"
143182,143182,10053207,29999444,31112026,transfer,Medical/Surgical Intensive Care Unit (MICU/SICU),2199-12-14 19:52:26,2199-12-17 23:22:57,93.333333,4.025000,...,ENGLISH,WIDOWED,WHITE,2199-12-14 13:18:00,2199-12-14 21:29:00,0.0,F,59.0,"['0389 ', '5845 ', '59010 ', '29680 ', '...","['720', '871']"
143183,143183,10053207,29999444,31230096,transfer,Med/Surg/GYN,2199-12-17 23:22:57,2199-12-19 13:00:07,82.000000,4.800000,...,ENGLISH,WIDOWED,WHITE,2199-12-14 13:18:00,2199-12-14 21:29:00,0.0,F,59.0,"['0389 ', '5845 ', '59010 ', '29680 ', '...","['720', '871']"
143184,143184,10053207,29999444,30263245,transfer,Med/Surg/GYN,2199-12-19 13:00:07,2199-12-22 14:02:54,94.000000,5.233333,...,ENGLISH,WIDOWED,WHITE,2199-12-14 13:18:00,2199-12-14 21:29:00,0.0,F,59.0,"['0389 ', '5845 ', '59010 ', '29680 ', '...","['720', '871']"


In [38]:
icu = list(hf_t.loc[hf_t["careunit"].str.contains("Care Unit")]["hadm_id"])

In [39]:
first_dep = hf_t.groupby("hadm_id").first()

In [40]:
hadms_f_d = list(first_dep.loc[first_dep["careunit"] == "Emergency Department"].reset_index()["hadm_id"])

In [44]:
hf_t_ed = hf_t.loc[hf_t["hadm_id"].isin(hadms_f_d)]

In [45]:
hf_t_ed = hf_t_ed.loc[hf_t["hadm_id"].isin(icu)]

In [46]:
hf_icu = hf_t_ed

In [47]:
hf_icu.loc[(hf_icu["careunit"]=="Emergency Department"), "department"] = "Emergency Department"

In [48]:
hf_icu.loc[(hf_icu["careunit"].str.contains("CCU")) | (hf_icu["careunit"].str.contains("CVICU")), "department"] = "Cardiac ICU"

In [49]:
hf_icu.loc[(hf_icu["careunit"].str.contains("SICU")), "department"] = "Surgical ICU"

In [50]:
hf_icu.loc[(hf_icu["careunit"].str.contains("MICU")) & (~hf_icu["careunit"].str.contains("SICU")), "department"] = "Medical ICU"

In [51]:
hf_icu.loc[(hf_icu["careunit"]== "Medicine"), "department"] = "Medicine"

In [52]:
hf_icu.loc[(hf_icu["careunit"]=="Medicine/Cardiology") | (hf_icu["careunit"]=="Medicine/Cardiology Intermediate"), "department"] = "Cardiology"

In [53]:
hf_icu.loc[(hf_icu["careunit"]=="Cardiology Surgery Intermediate") | (hf_icu["careunit"]=="Cardiology Surger") | (hf_icu["careunit"].str.contains("Med/Surg")), "department"] = "Surgery"

In [54]:
hf_icu.loc[(hf_icu["careunit"]== "Discharged"), "department"] = "Discharged"

In [55]:
to_remove = list(hf_icu.loc[hf_icu["department"].isna()]["hadm_id"].unique())

In [56]:
hf_icu_clean = hf_icu.loc[~hf_icu["hadm_id"].isin(to_remove)]

In [57]:
hf_icu_clean["transfer_duration"] = hf_icu_clean["outtime"] - hf_icu_clean["intime"]

C:\Users\jonas\AppData\Local\Temp\ipykernel_10128\675131562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hf_icu_clean["transfer_duration"] = hf_icu_clean["outtime"] - hf_icu_clean["intime"]


In [58]:
hf_icu_clean = hf_icu_clean.reset_index().sort_values(["hadm_id", "intime"]).drop("index", axis=1)

In [78]:
hf_icu_clean

,Unnamed: 0,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,Glucose_valuenum,Potassium_valuenum,...,case:race,case:edregtime,case:edouttime,case:hospital_expire_flag,case:gender,case:age,case:icd_code,case:drg_code,department,transfer_duration
0,34,12527107,20002636,34228746,ED,Emergency Department,2148-02-21 15:50:00,2148-02-21 19:32:00,NaN,NaN,...,WHITE,2148-02-21 15:50:00,2148-02-21 19:32:00,0.0,M,55.0,"['E860 ', 'N179 ', 'I428 ', 'I5022 ', '...","['422', '641']",Emergency Department,0 days 03:42:00
1,35,12527107,20002636,39096210,admit,Medicine/Cardiology,2148-02-21 19:32:00,2148-02-25 15:04:36,88.000000,4.360000,...,WHITE,2148-02-21 15:50:00,2148-02-21 19:32:00,0.0,M,55.0,"['E860 ', 'N179 ', 'I428 ', 'I5022 ', '...","['422', '641']",Cardiology,3 days 19:32:36
2,36,12527107,20002636,35028853,transfer,Cardiac Vascular Intensive Care Unit (CVICU),2148-02-25 15:04:36,2148-02-25 21:38:51,NaN,NaN,...,WHITE,2148-02-21 15:50:00,2148-02-21 19:32:00,0.0,M,55.0,"['E860 ', 'N179 ', 'I428 ', 'I5022 ', '...","['422', '641']",Cardiac ICU,0 days 06:34:15
3,37,12527107,20002636,37169762,transfer,Coronary Care Unit (CCU),2148-02-25 21:38:51,2148-02-28 17:57:16,110.600000,4.760000,...,WHITE,2148-02-21 15:50:00,2148-02-21 19:32:00,0.0,M,55.0,"['E860 ', 'N179 ', 'I428 ', 'I5022 ', '...","['422', '641']",Cardiac ICU,2 days 20:18:25
4,38,12527107,20002636,36566819,transfer,Medicine/Cardiology,2148-02-28 17:57:16,2148-03-01 17:28:12,89.500000,4.150000,...,WHITE,2148-02-21 15:50:00,2148-02-21 19:32:00,0.0,M,55.0,"['E860 ', 'N179 ', 'I428 ', 'I5022 ', '...","['422', '641']",Cardiology,1 days 23:30:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17999,143171,15477562,29997666,35092454,discharge,Discharged,2127-10-15 15:59:48,2127-10-15 15:59:49,NaN,NaN,...,WHITE,2127-10-11 20:20:00,2127-10-12 00:15:00,0.0,M,67.0,"['42843 ', '486 ', '5849 ', '4280 ', '...","['194', '291']",Discharged,0 days 00:00:01
18000,143175,16968104,29998702,36837654,ED,Emergency Department,2190-11-13 17:27:00,2190-11-13 22:42:00,NaN,NaN,...,WHITE,2190-11-13 17:27:00,2190-11-13 22:42:00,0.0,M,59.0,"['J690 ', 'E43 ', 'N179 ', 'C155 ', '...","['137', '177']",Emergency Department,0 days 05:15:00
18001,143176,16968104,29998702,37074618,admit,Medical Intensive Care Unit (MICU),2190-11-13 22:42:00,2190-11-15 08:21:09,100.333333,3.333333,...,WHITE,2190-11-13 17:27:00,2190-11-13 22:42:00,0.0,M,59.0,"['J690 ', 'E43 ', 'N179 ', 'C155 ', '...","['137', '177']",Medical ICU,1 days 09:39:09
18002,143177,16968104,29998702,38523336,transfer,Med/Surg,2190-11-15 08:21:09,2190-11-18 10:17:13,94.500000,3.600000,...,WHITE,2190-11-13 17:27:00,2190-11-13 22:42:00,0.0,M,59.0,"['J690 ', 'E43 ', 'N179 ', 'C155 ', '...","['137', '177']",Surgery,3 days 01:56:04


In [61]:
newdf = pd.DataFrame(data=None, columns=hf_icu_clean.columns)

In [62]:
hadms = hf_icu_clean["hadm_id"].unique()

In [65]:
d = {'subject_id':'first', 'hadm_id':'first', 'transfer_id':'first', 'eventtype':'first', 'careunit':'first', 'intime':'min', 'outtime':'max'}
for i in hadms:
    hadm = hf_icu_clean.loc[hf_icu_clean["hadm_id"] == i]
    consecutive_array = (hadm.department != hadm.department.shift()).cumsum().values
    new_hadm = hadm.groupby([consecutive_array, 'department']).agg(d).reset_index(level=1)
    newdf = pd.concat([newdf, new_hadm])
    
    

In [66]:
hf_consecutive = newdf

In [67]:
hadms = hf_consecutive["hadm_id"].unique()

In [68]:
hf_consecutive = hf_consecutive.reset_index().drop("index", axis=1)

In [69]:
hf_consecutive = hf_consecutive.reset_index()

In [70]:
hf_consecutive

,index,Unnamed: 0,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,Glucose_valuenum,...,case:race,case:edregtime,case:edouttime,case:hospital_expire_flag,case:gender,case:age,case:icd_code,case:drg_code,department,transfer_duration
0,0,NaN,12527107,20002636,34228746,ED,Emergency Department,2148-02-21 15:50:00,2148-02-21 19:32:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Department,NaN
1,1,NaN,12527107,20002636,39096210,admit,Medicine/Cardiology,2148-02-21 19:32:00,2148-02-25 15:04:36,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cardiology,NaN
2,2,NaN,12527107,20002636,35028853,transfer,Cardiac Vascular Intensive Care Unit (CVICU),2148-02-25 15:04:36,2148-02-28 17:57:16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cardiac ICU,NaN
3,3,NaN,12527107,20002636,36566819,transfer,Medicine/Cardiology,2148-02-28 17:57:16,2148-03-01 17:28:12,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cardiology,NaN
4,4,NaN,12527107,20002636,31751552,discharge,Discharged,2148-03-01 17:28:12,2148-03-01 17:28:13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Discharged,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16147,16147,NaN,15477562,29997666,35092454,discharge,Discharged,2127-10-15 15:59:48,2127-10-15 15:59:49,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Discharged,NaN
16148,16148,NaN,16968104,29998702,36837654,ED,Emergency Department,2190-11-13 17:27:00,2190-11-13 22:42:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Department,NaN
16149,16149,NaN,16968104,29998702,37074618,admit,Medical Intensive Care Unit (MICU),2190-11-13 22:42:00,2190-11-15 08:21:09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medical ICU,NaN
16150,16150,NaN,16968104,29998702,38523336,transfer,Med/Surg,2190-11-15 08:21:09,2190-11-18 10:17:13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Surgery,NaN


In [71]:
icu_flag = 0
for i in hadms:
    hadm = hf_consecutive.loc[hf_consecutive["hadm_id"] == i]
    for index, row in hadm.iterrows():
        r = row
        if("ICU" in row["department"]):
            icu_flag = 1
        if((row["department"] != "Emergency Department") & (row["department"] != "Discharged") & ("ICU" not in row["department"])):
            if(icu_flag == 1):
                hf_consecutive.at[row["index"], "department"] = "Post-ICU " + row["department"] 
            else:
                hf_consecutive.at[row["index"], "department"] = "Pre-ICU " + row["department"] 
    icu_flag = 0

In [72]:
hf_consecutive["transfer_duration"] = hf_consecutive["outtime"] - hf_consecutive["intime"]

In [73]:
hadms = list(hf_consecutive["hadm_id"].unique())

In [74]:
#consider only patients which have been in the ICU once
number_of_icu_visits = hf_consecutive.loc[hf_consecutive["department"].str.contains('Cardiac ICU|Medical ICU|Surgical ICU')].groupby("hadm_id").count().reset_index()

In [75]:
to_remove = list(number_of_icu_visits.loc[number_of_icu_visits["subject_id"] > 1]["hadm_id"])

In [76]:
hf_consecutive = hf_consecutive.loc[~hf_consecutive["hadm_id"].isin(to_remove)]

In [81]:
hf_consecutive = hf_consecutive[['subject_id', 'hadm_id', 'transfer_id', 'eventtype', 'careunit', 'intime', 'outtime', 'department', 'transfer_duration']]

In [91]:
to_merge = hf_icu_clean.drop(["Unnamed: 0", "department", "transfer_duration"], axis=1)

In [95]:
final_df = hf_consecutive.merge(to_merge, how="left")

In [ ]:
hf_consecutive.to_csv("Kidney_Failure_Log.csv")